# Testing - Imitation
In this code, we aim to demonstrate how accurately the RDR model created by the proxy replicates the performance of the original model. By evaluating the accuracy, we can assess whether the explanations provided by the RDR model effectively represent the behavior of the original model.

In [1]:
import sys
sys.path.append('../src/')
from rdr import RDR

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Import Model Libraries

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from pytorch_tabnet.tab_model import TabNetClassifier

## 1. Load dataset

### 1.1. Load the dataset and store it in a variable named **df**

In [4]:
df = pd.read_csv('../data/star_classification.csv')
display(df.head())
display(df.info())

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   obj_ID       100000 non-null  float64
 1   alpha        100000 non-null  float64
 2   delta        100000 non-null  float64
 3   u            100000 non-null  float64
 4   g            100000 non-null  float64
 5   r            100000 non-null  float64
 6   i            100000 non-null  float64
 7   z            100000 non-null  float64
 8   run_ID       100000 non-null  int64  
 9   rerun_ID     100000 non-null  int64  
 10  cam_col      100000 non-null  int64  
 11  field_ID     100000 non-null  int64  
 12  spec_obj_ID  100000 non-null  float64
 13  class        100000 non-null  object 
 14  redshift     100000 non-null  float64
 15  plate        100000 non-null  int64  
 16  MJD          100000 non-null  int64  
 17  fiber_ID     100000 non-null  int64  
dtypes: float64(10), int64(7),

None

### 1.2. Define and train Label Encoder

In [5]:
le = LabelEncoder()
le.fit(df['class'])

LabelEncoder()

## 2. Testing RDR model's imitation accuracy
At this phase, the RDR model will be created and compared against the original models. The conventional models to be used consists of:

**White box Model**
- Decision Tree Classifier

**Black box Model**
- Random Forest
- Support Vector Machine
- XGBoost
- MLP
- TabNet

Furthermore, testing will be carried out on data samples which differs from 25%, 50%, 75%, and 100% (its original size).

### 2.4. Testing with sample data size : 100%

In [6]:
def remove_outliers(df, threshold=2.5):
    z_scores = np.abs(stats.zscore(df.select_dtypes(include='number')))
    outliers = (z_scores > threshold).any(axis=1)
    return df[~outliers]

# Sample size : 100%
df100 = df

# remove outlier
df100 = remove_outliers(df100)

# define features and label
label = df100['class']
features = df100.drop(
            columns=['obj_ID', 'run_ID', 'rerun_ID', 'field_ID', 'spec_obj_ID','fiber_ID','class'],
            axis=1)

# split data for training and testing model
# train : test = 80 : 20
X100_train, X100_test, y100_train, y100_test = train_test_split(features,
                                                            label, 
                                                            test_size=0.2,
                                                            random_state=42)

#### 2.4.1. RDR against Decision Tree Classifier

In [7]:
# CREATE AND TRAIN DECISION TREE CLASSIFIER
dtf100 = DecisionTreeClassifier()
dtf100.fit(X100_train, y100_train)

DecisionTreeClassifier()

In [8]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = dtf100.predict(features)

In [9]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

dtf_rdr100 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
dtf100_pred = dtf100.predict(X100_test)
dtf_rdr100_pred = dtf_rdr100.predict(X100_test)

#### 2.4.2. RDR against Random Forest Classifier

In [10]:
# CREATE AND TRAIN RANDOM FOREST CLASSIFIER
rf100 = RandomForestClassifier()
rf100.fit(X100_train, y100_train)

RandomForestClassifier()

In [11]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = rf100.predict(features)

In [12]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

rf_rdr100 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
rf100_pred = rf100.predict(X100_test)
rf_rdr100_pred = rf_rdr100.predict(X100_test)

#### 2.4.3. RDR against Support Vector Machine

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear' 'poly', 'sigmoid']}

svm = SVC()

grid = GridSearchCV(svm,param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid.fit(X100_train, y100_train)

print("Best parameters found: ", grid.best_params_)
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))

In [13]:
# CREATE AND TRAIN SUPPORT VECTOR MACHINE
svm100 = SVC(C= 100, gamma= 0.001, kernel= 'rbf')
svm100.fit(X100_train, y100_train)

SVC(C=100, gamma=0.001)

In [14]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = svm100.predict(features)

In [15]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

svm_rdr100 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
svm100_pred = svm100.predict(X100_test)
svm_rdr100_pred = svm_rdr100.predict(X100_test)

#### 2.4.4. RDR against XGBoost Classifier

In [16]:
# CREATE AND TRAIN XGBOOST CLASSIFIER
train_dmatrix = xgb.DMatrix(data=X100_train, label=le.transform(y100_train))
xgboost = xgb.train({}, train_dmatrix)

In [17]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
dmatrix = xgb.DMatrix(
            data=features,
            label=le.transform(label))

xgb100_pred = xgboost.predict(dmatrix)
prediction_dataset = le.inverse_transform(np.round(xgb100_pred).astype(int))

In [18]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

xgb_rdr100 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
dmatrix = xgb.DMatrix(
            data=X100_test,
            label=le.transform(y100_test))

xgb100_pred = xgboost.predict(dmatrix)
xgb100_pred = le.inverse_transform(np.round(xgb100_pred).astype(int))

xgb_rdr100_pred = xgb_rdr100.predict(X100_test)

#### 2.4.5. RDR against Multi-layer Perceptron (MLP) Classifier

In [49]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.1, 0.01, 0.001],
    'batch_size': [32, 64],
    'max_iter': [300, 500],
    'alpha': [0.0001, 0.001],
    'activation': ['relu', 'tanh']
}

mlp = MLPClassifier()

grid_search = GridSearchCV(mlp, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X100_train, y100_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: {:.2f}".format(grid_search.best_score_))

Best parameters found:  {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 32, 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 300}
Best cross-validation accuracy: 0.91


In [19]:
# CREATE AND TRAIN MLP CLASSIFIER
mlp100 = MLPClassifier(
    max_iter=300,
    batch_size=32,
    hidden_layer_sizes=(50,),
    learning_rate='adaptive',
    learning_rate_init=0.001,
    alpha=0.0001,
    activation='relu'
)
mlp100.fit(X100_train, y100_train)

MLPClassifier(batch_size=32, hidden_layer_sizes=(50,), learning_rate='adaptive',
              max_iter=300)

In [20]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = mlp100.predict(features)

In [21]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

mlp_rdr100 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT - MLP AND RDR
mlp100_pred = mlp100.predict(X100_test)
mlp_rdr100_pred = mlp_rdr100.predict(X100_test)

#### 2.4.6. RDR against TabNet Classifier

In [22]:
# CREATE AND TRAIN TabNet Classifier
tabnet100 = TabNetClassifier()
tabnet100.fit(X100_train.values, y100_train, max_epochs=100)

c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.4646  |  0:00:03s
epoch 1  | loss: 0.14333 |  0:00:06s
epoch 2  | loss: 0.12653 |  0:00:10s
epoch 3  | loss: 0.11649 |  0:00:13s
epoch 4  | loss: 0.11097 |  0:00:17s
epoch 5  | loss: 0.10862 |  0:00:20s
epoch 6  | loss: 0.10425 |  0:00:23s
epoch 7  | loss: 0.10446 |  0:00:27s
epoch 8  | loss: 0.10487 |  0:00:30s
epoch 9  | loss: 0.10255 |  0:00:34s
epoch 10 | loss: 0.10099 |  0:00:37s
epoch 11 | loss: 0.09989 |  0:00:41s
epoch 12 | loss: 0.09955 |  0:00:44s
epoch 13 | loss: 0.10122 |  0:00:48s
epoch 14 | loss: 0.09926 |  0:00:51s
epoch 15 | loss: 0.09935 |  0:00:55s
epoch 16 | loss: 0.09798 |  0:00:58s
epoch 17 | loss: 0.0973  |  0:01:01s
epoch 18 | loss: 0.10043 |  0:01:05s
epoch 19 | loss: 0.09942 |  0:01:08s
epoch 20 | loss: 0.09739 |  0:01:12s
epoch 21 | loss: 0.09549 |  0:01:15s
epoch 22 | loss: 0.0963  |  0:01:19s
epoch 23 | loss: 0.09518 |  0:01:22s
epoch 24 | loss: 0.09324 |  0:01:26s
epoch 25 | loss: 0.09403 |  0:01:29s
epoch 26 | loss: 0.09357 |  0:01:33s
e

In [23]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = tabnet100.predict(features.values)

In [24]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

tabnet_rdr100 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT - TabNet AND RDR
tabnet100_pred = tabnet100.predict(X100_test.values)
tabnet_rdr100_pred = tabnet_rdr100.predict(X100_test)

## 5. Accuracy Comparison between RDR model and conventional models
To streamline the process, the accuracy comparison for each sample size will be presented by invoking the `display_accuracy` function.

In [25]:
def display_accuracy(
        y_test,
        dtf_pred, dtf_rdr_pred,
        rf_pred, rf_rdr_pred,
        xgb_pred, xgb_rdr_pred,
        svm_pred, svm_rdr_pred,
        mlp_pred, mlp_rdr_pred,
        tabnet_pred, tabnet_rdr_pred
    ) -> None:

    accuracy_table = []

    accuracy_table.append([
        "Decision Tree",
        metrics.accuracy_score(y_test,dtf_pred),
        metrics.accuracy_score(y_test, dtf_rdr_pred),
        metrics.accuracy_score(dtf_pred, dtf_rdr_pred)
    ])

    accuracy_table.append([
        "Random Forest",
        metrics.accuracy_score(y_test,rf_pred),
        metrics.accuracy_score(y_test, rf_rdr_pred),
        metrics.accuracy_score(rf_pred, rf_rdr_pred)
    ])

    accuracy_table.append([
        "XGBoost",
        metrics.accuracy_score(y_test,xgb_pred),
        metrics.accuracy_score(y_test, xgb_rdr_pred),
        metrics.accuracy_score(xgb_pred, xgb_rdr_pred)
    ])

    accuracy_table.append([
        "Support Vector Machine",
        metrics.accuracy_score(y_test,svm_pred),
        metrics.accuracy_score(y_test, svm_rdr_pred),
        metrics.accuracy_score(svm_pred, svm_rdr_pred)
    ])

    accuracy_table.append([
        "Multi-layer Perceptron",
        metrics.accuracy_score(y_test,mlp_pred),
        metrics.accuracy_score(y_test, mlp_rdr_pred),
        metrics.accuracy_score(mlp_pred, mlp_rdr_pred)
    ])

    accuracy_table.append([
        "TabNet",
        metrics.accuracy_score(y_test,tabnet_pred),
        metrics.accuracy_score(y_test, tabnet_rdr_pred),
        metrics.accuracy_score(tabnet_pred, tabnet_rdr_pred)
    ])

    accuracy_table = pd.DataFrame(accuracy_table, columns=["Model", "Accuracy", "RDR Accuracy", "Difference"])
    pd.set_option('display.precision', 15)

    display(accuracy_table)

    return

In [26]:
print("Accuracy comparison for 100% sample size:")
display_accuracy(
    y100_test,
    dtf100_pred, dtf_rdr100_pred,
    rf100_pred, rf_rdr100_pred,
    xgb100_pred, xgb_rdr100_pred,
    svm100_pred, svm_rdr100_pred,
    mlp100_pred, mlp_rdr100_pred,
    tabnet100_pred, tabnet_rdr100_pred
)

Accuracy comparison for 100% sample size:


,Model,Accuracy,RDR Accuracy,Difference
0,Decision Tree,0.965447927418917,0.876025425110012,0.890911066442114
1,Random Forest,0.977617210843701,0.898299559950019,0.913293855598414
2,XGBoost,0.970065735861357,0.867604715597327,0.884391807464551
3,Support Vector Machine,0.881349486608356,0.809528983538871,0.868256641495083
4,Multi-layer Perceptron,0.826805019829413,0.761558102895638,0.895039930461238
5,TabNet,0.762590318900418,0.730102678328897,0.952626718096376
